In [ ]:
import numpy as np
from scipy.sparse import dok_matrix

In [ ]:
lines=[]
with open('david_of_sassoun.txt','r',encoding='utf-8') as f:
    lines.extend(f.readlines())

In [ ]:
new_lines=[]
all_text=''
for i,line in enumerate(lines):
    text=line.replace('\n','')
    if len(text)>3 and '[' not in text and ']' not in text:
        new_lines.append(text)

for line in new_lines:
    all_text=all_text+' <START> '+line+ ' <END>'

for spaced in ['.','-',',','!','?','(','—',')','՞','՛','։','՝','՜','’','«','»','*']:
    all_text = all_text.replace(spaced, ' {0} '.format(spaced))

all_text=all_text.replace('   ',' ')
all_text=all_text.replace('  ',' ')
all_text=all_text[1:]

all_text

сглаживание 4 лекция

In [ ]:
corpus = ""
for file_name in file_names:
    with open(file_name, 'r') as f:
        corpus+=f.read()
corpus = corpus.replace('\n',' ')
corpus = corpus.replace('\t',' ')
corpus = corpus.replace('“', ' " ')
corpus = corpus.replace('”', ' " ')
for spaced in ['.','-',',','!','?','(','—',')']:
    corpus = corpus.replace(spaced, ' {0} '.format(spaced))
len(corpus) #10510355 characters

In [ ]:
corpus_words = corpus.split(' ')
corpus_words= [word for word in corpus_words if word != '']
corpus_words # [...'a', 'wyvern', ',', 'two', 'of', 'the', 'thousand'...]
len(corpus_words) # 2185920

In [ ]:
distinct_words = list(set(corpus_words))
word_idx_dict = {word: i for i, word in enumerate(distinct_words)}
distinct_words_count = len(list(set(corpus_words)))
distinct_words_count # 32663

In [ ]:
k = 2 # adjustable
sets_of_k_words = [ ' '.join(corpus_words[i:i+k]) for i, _ in enumerate(corpus_words[:-k]) ]

sets_count = len(list(set(sets_of_k_words)))
next_after_k_words_matrix = dok_matrix((sets_count, len(distinct_words)))

distinct_sets_of_k_words = list(set(sets_of_k_words))
k_words_idx_dict = {word: i for i, word in enumerate(distinct_sets_of_k_words)}

for i, word in enumerate(sets_of_k_words[:-k]):

    word_sequence_idx = k_words_idx_dict[word]
    next_word_idx = word_idx_dict[corpus_words[i+k]]
    next_after_k_words_matrix[word_sequence_idx, next_word_idx] +=1


In [ ]:
def sample_next_word_after_sequence(word_sequence, alpha = 0):
    next_word_vector = next_after_k_words_matrix[k_words_idx_dict[word_sequence]] + alpha
    likelihoods = next_word_vector/next_word_vector.sum()

    return np.random.choice(distinct_words, p=likelihoods.toarray())

def stochastic_chain(seed, chain_length=15, seed_length=2):
    current_words = seed.split(' ')
    if len(current_words) != seed_length:
        raise ValueError(f'wrong number of words, expected {seed_length}')
    sentence = seed

    for _ in range(chain_length):
        sentence+=' '
        next_word = sample_next_word_after_sequence(' '.join(current_words))
        sentence+=next_word
        current_words = current_words[1:]+[next_word]
    return sentence


In [ ]:
# example use
stochastic_chain('the world')